## 1. Setup & Load Features

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler

# Time series models
try:
    from fbprophet import Prophet
    print("✓ Prophet imported")
except ImportError:
    print("⚠ Prophet not installed. Install with: pip install pystan==2.19.1.1 fbprophet")

from statsmodels.tsa.arima.model import ARIMA
from xgboost import XGBRegressor

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ All libraries imported successfully")

In [ ]:
# Load engineered features
try:
    df = pd.read_csv('../data/processed/engineered_features.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').reset_index(drop=True)
    print(f"✓ Loaded engineered features: {df.shape[0]} rows, {df.shape[1]} columns")
except FileNotFoundError:
    print("⚠ Engineered features not found. Please run feature_engineering notebook first.")

## 2. Train-Test Split

In [ ]:
# Split data: 80% train, 20% test
split_date = df['date'].quantile(0.8)
df_train = df[df['date'] <= split_date].reset_index(drop=True)
df_test = df[df['date'] > split_date].reset_index(drop=True)

print(f"Training set: {len(df_train)} samples ({split_date.date()})")
print(f"Test set: {len(df_test)} samples")
print(f"\nTrain-Test Split Ratio: {len(df_train)/len(df):.1%} / {len(df_test)/len(df):.1%}")

# Visualize split
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(df_train['date'], df_train['sales'], label='Training Data', color='steelblue', linewidth=1.5)
ax.plot(df_test['date'], df_test['sales'], label='Test Data', color='coral', linewidth=1.5)
ax.axvline(split_date, color='red', linestyle='--', alpha=0.7, label='Train-Test Split')
ax.set_title('Train-Test Split', fontsize=12, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Sales')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/04_train_test_split.png', dpi=300, bbox_inches='tight')
plt.show()
print("\n✓ Saved: 04_train_test_split.png")

## 3. Model 1: Facebook Prophet

In [ ]:
# Prepare data for Prophet (requires 'ds' and 'y' columns)
prophet_train = df_train[['date', 'sales']].copy()
prophet_train.columns = ['ds', 'y']

# Initialize and train Prophet
try:
    print("Training Prophet model...")
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_prior_scale=10,
        interval_width=0.95
    )
    
    # Add regressors if available
    if 'is_weekend' in df_train.columns:
        prophet_train['is_weekend'] = df_train['is_weekend'].values
        prophet_model.add_regressor('is_weekend')
    
    prophet_model.fit(prophet_train)
    print("✓ Prophet model trained successfully")
    
    # Make predictions on test set
    prophet_test = df_test[['date', 'sales']].copy()
    prophet_test.columns = ['ds', 'y']
    if 'is_weekend' in df_train.columns:
        prophet_test['is_weekend'] = df_test['is_weekend'].values
    
    prophet_forecast = prophet_model.make_future_dataframe(periods=len(df_test))
    if 'is_weekend' in df_train.columns:
        prophet_forecast = prophet_forecast.merge(df[['date', 'is_weekend']], left_on='ds', right_on='date', how='left')
        prophet_forecast = prophet_forecast.drop('date', axis=1)
    
    prophet_forecast = prophet_model.make_future_dataframe(periods=0)
    prophet_forecast_test = prophet_model.predict(prophet_forecast.iloc[-len(df_test):])
    
    prophet_pred = prophet_forecast_test['yhat'].values
    print(f"✓ Prophet predictions made for {len(prophet_pred)} test samples")
    
except Exception as e:
    print(f"⚠ Prophet training error: {str(e)}")
    prophet_pred = None

## 4. Model 2: ARIMA

In [ ]:
# Train ARIMA model
try:
    print("Training ARIMA model...")
    # ARIMA(p,d,q) - commonly used: (5,1,2)
    arima_model = ARIMA(df_train['sales'], order=(5, 1, 2))
    arima_result = arima_model.fit()
    print("✓ ARIMA model trained successfully")
    print(f"\nARIMA Summary:")
    print(arima_result.summary())
    
    # Make predictions on test set
    arima_pred = arima_result.get_forecast(steps=len(df_test)).predicted_mean.values
    print(f"✓ ARIMA predictions made for {len(arima_pred)} test samples")

except Exception as e:
    print(f"⚠ ARIMA training error: {str(e)}")
    arima_pred = None

## 5. Model 3: XGBoost

In [ ]:
# Prepare features for XGBoost
# Select relevant features (excluding date and target)
feature_cols = [col for col in df_train.columns if col not in ['date', 'sales']]

X_train = df_train[feature_cols].fillna(0)
y_train = df_train['sales']
X_test = df_test[feature_cols].fillna(0)
y_test = df_test['sales']

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"XGBoost Features: {len(feature_cols)}")
print(f"Training samples: {len(X_train_scaled)}")
print(f"Test samples: {len(X_test_scaled)}")

In [ ]:
# Train XGBoost
try:
    print("Training XGBoost model...")
    xgb_model = XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        verbose=0
    )
    
    xgb_model.fit(
        X_train_scaled, y_train,
        eval_set=[(X_test_scaled, y_test)],
        early_stopping_rounds=10,
        verbose=False
    )
    
    print("✓ XGBoost model trained successfully")
    
    # Make predictions
    xgb_pred = xgb_model.predict(X_test_scaled)
    print(f"✓ XGBoost predictions made for {len(xgb_pred)} test samples")
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Important Features:")
    print(feature_importance.head(10))

except Exception as e:
    print(f"⚠ XGBoost training error: {str(e)}")
    xgb_pred = None

## 6. Model Evaluation & Comparison

In [ ]:
# Calculate metrics for all models
def calculate_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    return {
        'Model': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R²': r2
    }

results = []

# Evaluate each model
if prophet_pred is not None:
    results.append(calculate_metrics(df_test['sales'], prophet_pred, 'Prophet'))

if arima_pred is not None:
    # Ensure predictions are positive (sales shouldn't be negative)
    arima_pred_positive = np.maximum(arima_pred, 0)
    results.append(calculate_metrics(df_test['sales'], arima_pred_positive, 'ARIMA'))

if xgb_pred is not None:
    results.append(calculate_metrics(df_test['sales'], xgb_pred, 'XGBoost'))

# Create comparison dataframe
results_df = pd.DataFrame(results)
print("\n=== MODEL COMPARISON ===")
print(results_df.to_string(index=False))

# Find best model
best_model = results_df.loc[results_df['R²'].idxmax(), 'Model']
print(f"\n✓ Best Model (by R² score): {best_model}")

In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

models = [
    ('Prophet', prophet_pred, axes[0]),
    ('ARIMA', np.maximum(arima_pred, 0) if arima_pred is not None else None, axes[1]),
    ('XGBoost', xgb_pred, axes[2])
]

for model_name, predictions, ax in models:
    if predictions is not None:
        ax.plot(df_test['date'], df_test['sales'], label='Actual', color='steelblue', linewidth=2, marker='o', markersize=4)
        ax.plot(df_test['date'], predictions, label='Predicted', color='coral', linewidth=2, marker='s', markersize=4, alpha=0.7)
        
        # Calculate error for this model
        mae = mean_absolute_error(df_test['sales'], predictions)
        rmse = np.sqrt(mean_squared_error(df_test['sales'], predictions))
        
        ax.set_title(f'{model_name} Predictions (MAE: ${mae:.2f}, RMSE: ${rmse:.2f})', fontsize=12, fontweight='bold')
        ax.set_ylabel('Sales')
        ax.legend(loc='upper left')
        ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.savefig('../outputs/05_model_predictions.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved: 05_model_predictions.png")

In [ ]:
# Comparison chart
fig, axes = plt.subplots(1, 4, figsize=(16, 5))

metrics = ['MAE', 'RMSE', 'MAPE', 'R²']
for i, metric in enumerate(metrics):
    axes[i].barh(results_df['Model'], results_df[metric], color=['steelblue', 'coral', 'lightgreen'][:len(results_df)])
    axes[i].set_title(metric, fontsize=12, fontweight='bold')
    axes[i].set_xlabel(metric)
    
    # Add value labels
    for j, v in enumerate(results_df[metric]):
        axes[i].text(v + 0.01, j, f'{v:.4f}', va='center')

plt.tight_layout()
plt.savefig('../outputs/06_model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved: 06_model_comparison.png")

## 7. Feature Importance (XGBoost)

In [ ]:
if xgb_pred is not None:
    # Plot feature importance
    fig, ax = plt.subplots(figsize=(12, 8))
    
    top_features = feature_importance.head(15)
    ax.barh(range(len(top_features)), top_features['importance'].values, color='steelblue')
    ax.set_yticks(range(len(top_features)))
    ax.set_yticklabels(top_features['feature'].values)
    ax.set_xlabel('Feature Importance Score', fontsize=11, fontweight='bold')
    ax.set_title('Top 15 Most Important Features (XGBoost)', fontsize=12, fontweight='bold')
    ax.invert_yaxis()
    
    # Add value labels
    for i, v in enumerate(top_features['importance'].values):
        ax.text(v + 0.002, i, f'{v:.4f}', va='center')
    
    plt.tight_layout()
    plt.savefig('../outputs/07_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("✓ Saved: 07_feature_importance.png")

## 8. Save Models & Results

In [ ]:
# Save results to JSON
import json

results_dict = results_df.to_dict('records')
with open('../outputs/model_comparison_results.json', 'w') as f:
    json.dump(results_dict, f, indent=2)

print("✓ Saved: model_comparison_results.json")

# Save predictions to CSV
predictions_df = pd.DataFrame({
    'date': df_test['date'],
    'actual_sales': df_test['sales']